# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

Using TensorFlow backend.


### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 11616288872588114347
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 357433344
locality {
  bus_id: 1
}
incarnation: 12589441834804021932
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0"
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

num_encoder_tokens = len(english_words_counter)
num_decoder_tokens = len(french_words_counter)
max_encoder_seq_length = max([len(txt) for txt in english_sentences])
max_decoder_seq_length = max([len(txt) for txt in french_sentences])

print()
print('Number of samples:', len(english_sentences))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"

Number of samples: 137861
Number of unique input tokens: 227
Number of unique output tokens: 355
Max sequence length for inputs: 102
Max sequence length for outputs: 114


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    
    return x_tk.texts_to_sequences(x), x_tk

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
#     print (length)
    numpy_array = pad_sequences(x, maxlen=length, padding='post')
#     print(numpy_array)
    return numpy_array

tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [10]:
from keras.layers import GRU, Input, Dense, TimeDistributed,LSTM
from keras.models import Model
from keras.layers import Activation
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.layers import Bidirectional

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [11]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [86]:


def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y    
    
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    #-- Hyperparameters
    learning_rate = 0.01
    opt = Adam(lr=learning_rate, decay=1e-6)
    
    # -- Model Architecture 
    inputs = Input(shape=input_shape[1:])
    hidden_layer = LSTM(32, return_sequences=True)(inputs)
    outputs = TimeDistributed(Dense(french_vocab_size))(hidden_layer)
    
    # #     a = Input(shape=input_shape[1:])
    # #     h = Dense(64, activation='softmax')(a)
    # #     b = Dense(output_sequence_length, activation='softmax')(h)
    
    
    #setup the model 
    model = Model(inputs=inputs, outputs=outputs)
#     model = Model(inputs=a, outputs=b)

    model.compile(loss=sparse_categorical_crossentropy, optimizer=opt, metrics=['accuracy'])

    return model

tests.test_simple_model(simple_model)


# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))
print(tmp_x.shape)
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size+1,
    french_vocab_size+1)


simple_rnn_model.summary()
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=3, validation_split=0.2)

(137861, 21, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_66 (InputLayer)        (None, 21, 1)             0         
_________________________________________________________________
lstm_51 (LSTM)               (None, 21, 32)            4352      
_________________________________________________________________
time_distributed_27 (TimeDis (None, 21, 345)           11385     
Total params: 15,737
Trainable params: 15,737
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/3
110288/110288 [==============================] - 13s 118us/step - loss: 5.1051 - acc: 0.4216 - val_loss: 5.3808 - val_acc: 0.3735
Epoch 2/3
110288/110288 [==============================] - 6s 59us/step - loss: 4.1799 - acc: 0.4086 - val_loss: 4.1705 - val_acc: 0.3248
Epoch 3/3
110288/110288 [==============================] 

In [87]:
index_to_words = {id: word for word, id in english_tokenizer.word_index.items()}
index_to_words[0] = ''
sentence = [index_to_words.get(int(Ids)) for Ids in tmp_x[:1][0] if index_to_words.get(int(Ids))  is not None]
print ("English Sentence: ",' '.join(sentence))

# Print prediction(s)
print("French Sentence:" , logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

English Sentence:  new jersey is sometimes quiet during autumn and it is snowy in april        
French Sentence: lapins trop rouillée gros vert gros vert gros des il vert redoutée vert redoutée juillet <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [43]:
# from keras.models import Sequential


# from gensim.models import Word2Vec
# import multiprocessing 

def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
# acc: 0.7421
#     input_layer = Input(shape=input_shape[1:])
#     rnn = Embedding(english_vocab_size, french_vocab_size, input_length=output_sequence_length)(input_layer)
#     rnn = LSTM(french_vocab_size, return_sequences=True)(rnn)
#     rnn = Dense(french_vocab_size, activation='softmax')(rnn)

# # acc: 0.5444  - incremental validation
#     input_layer = Input(shape=input_shape[1:])
#     rnn = Embedding(english_vocab_size,max_english_sequence_length, input_length=output_sequence_length)(input_layer)
#     rnn = GRU(french_vocab_size, return_sequences=True)(rnn)
#     rnn = Dense(french_vocab_size, activation='softmax')(rnn)

# acc: 0.8010
    input_layer = Input(shape=input_shape[1:])
    rnn = Embedding(english_vocab_size, french_vocab_size, input_length=output_sequence_length)(input_layer)
    rnn = GRU(french_vocab_size, return_sequences=True)(rnn)
    rnn = Dense(french_vocab_size, activation='softmax')(rnn)

# # acc: 0.8255 
#     input_layer = Input(shape=input_shape[1:])
#     rnn = Embedding(english_vocab_size, french_vocab_size, input_length=output_sequence_length)(input_layer)
#     rnn = GRU(512, return_sequences=True)(rnn)
#     rnn = Dense(french_vocab_size, activation='softmax')(rnn)

    bd_model = Model(inputs=input_layer, outputs=rnn)
    bd_model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(.001), metrics=['accuracy'])

    
    return bd_model


tests.test_embed_model(embed_model)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# tmp_x = pad(preproc_english_sentences, max_english_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-1]))

# Train the neural network
ebd_model = embed_model( tmp_x.shape,max_french_sequence_length,english_vocab_size,french_vocab_size)

ebd_model.summary()

# TODO: Train the neural network
ebd_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=3, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (None, 21)                0         
_________________________________________________________________
embedding_32 (Embedding)     (None, 21, 344)           68456     
_________________________________________________________________
gru_32 (GRU)                 (None, 21, 344)           711048    
_________________________________________________________________
dense_31 (Dense)             (None, 21, 344)           118680    
Total params: 898,184
Trainable params: 898,184
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/3
110288/110288 [==============================] - 20s 177us/step - loss: 2.7836 - acc: 0.4824 - val_loss: nan - val_acc: 0.5949
Epoch 2/3
110288/110288 [==============================] - 18s 161us/step - loss

In [89]:
# sentence = [index_to_words.get(int(Ids)) for Ids in tmp_x[:1][0] if index_to_words.get(int(Ids))  is not None]
print ("English Sentence: ",' '.join(sentence))

# Print prediction(s)
print("French Sentence:" , logits_to_text(ebd_model.predict(tmp_x[:1])[0], french_tokenizer))

English Sentence:  new jersey is sometimes quiet during autumn and it is snowy in april        
French Sentence: new jersey est parfois calme en l'automne et il est il en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [90]:

def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """


# -------------------------------------
    input_layer = Input(shape=input_shape[1:])    
    
    bd = Embedding(max_english_sequence_length, english_vocab_size, input_length=output_sequence_length, trainable=False,name = 'layer_embedding')(input_layer)

    #https://stackoverflow.com/questions/49855404/keras-bidirectional-lstm-layer-grouping
#     rnn = Bidirectional(LSTM(french_vocab_size//2, return_sequences=True, dropout=0.25, recurrent_dropout=0.1),merge_mode='sum')(bd)

#     rnn = TimeDistributed(Dense(french_vocab_size))(bd)
    
    rnn = Bidirectional(LSTM(french_vocab_size//2, return_sequences=True, dropout=0.25, recurrent_dropout=0.1),merge_mode='concat')(bd)

#     rnn = Dense(french_vocab_size, activation='softmax')(rnn)

    rnn = GRU(french_vocab_size, return_sequences=True)(rnn)
    rnn = Dense(french_vocab_size, activation='softmax')(rnn)
        
    model = Model(inputs=input_layer, outputs=rnn)
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# ------------------------------------
              
    return model

# tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# print(tmp_x.shape)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]),)
# print(tmp_x.shape)

# print("max_french_sequence_length,english_vocab_size,french_vocab_size")
# print(max_french_sequence_length,english_vocab_size,french_vocab_size)

# print(max_english_sequence_length)

# tests.test_bd_model(bd_model)



# TODO: Train and Print prediction(s)
bid_model = bd_model( tmp_x.shape,max_french_sequence_length,english_vocab_size+1,french_vocab_size+1)

bid_model.summary()

# TODO: Train the neural network
bid_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=3, validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_69 (InputLayer)        (None, 21)                0         
_________________________________________________________________
layer_embedding (Embedding)  (None, 21, 200)           3000      
_________________________________________________________________
bidirectional_50 (Bidirectio (None, 21, 344)           513248    
_________________________________________________________________
gru_56 (GRU)                 (None, 21, 345)           714150    
_________________________________________________________________
dense_94 (Dense)             (None, 21, 345)           119370    
Total params: 1,349,768
Trainable params: 1,346,768
Non-trainable params: 3,000
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/3
110288/110288 [==============================] - 42s 384us/step - l

In [91]:
# sentence = [index_to_words.get(int(Ids)) for Ids in tmp_x[:1][0] if index_to_words.get(int(Ids))  is not None]
print ("English Sentence: ",' '.join(sentence))

# Print prediction(s)
print("French Sentence:" , logits_to_text(bid_model.predict(tmp_x[:1])[0], french_tokenizer))

English Sentence:  new jersey is sometimes quiet during autumn and it is snowy in april        
French Sentence: new jersey est parfois agréable en mois et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [92]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['<']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == ' .' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [93]:
# from keras.utils.vis_utils import plot_model
start = "<"
# end = ">"
modified_french_sentences = list(map(lambda x:"{}{}".format(start ,x), french_sentences))
# print(modified_french_sentences[0])

# Vectorize the data.
input_texts = english_sentences
target_texts = modified_french_sentences
input_characters = set()
target_characters = set()

for input_text in input_texts:
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
for target_text in target_texts:
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)


input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
            

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

# start = 'START_ '
# end = ' _END'
# modified_french_sentences = list(map(lambda x:"{}{}{}".format(start ,x, end), french_sentences))
# # print(modified_french_sentences[0])

#hyperparameters
latent_dim = 256

# Define an input sequence and process it
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(optimizer=Adam(0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(0.001), metrics=['accuracy'])

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=1024,
          epochs=3,
          validation_split=0.2)

# plot_model(model, to_file='model.png', show_shapes=True)

encoder_model = Model(encoder_inputs, encoder_states)

# define decoder inference model
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

Number of samples: 137861
Number of unique input tokens: 31
Number of unique output tokens: 41
Max sequence length for inputs: 102
Max sequence length for outputs: 115
Train on 110288 samples, validate on 27573 samples
Epoch 1/3
110288/110288 [==============================] - 105s 950us/step - loss: 1.6821 - acc: 0.5148 - val_loss: 1.3856 - val_acc: 0.6029
Epoch 2/3
110288/110288 [==============================] - 97s 881us/step - loss: 1.0902 - acc: 0.6474 - val_loss: 0.8637 - val_acc: 0.5413
Epoch 3/3
110288/110288 [==============================] - 97s 882us/step - loss: 0.6516 - acc: 0.5912 - val_loss: 0.4984 - val_acc: 0.5867


In [94]:
# def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
#     """
#     Build and train an encoder-decoder model on x and y
#     :param input_shape: Tuple of input shape
#     :param output_sequence_length: Length of output sequence
#     :param english_vocab_size: Number of unique English words in the dataset
#     :param french_vocab_size: Number of unique French words in the dataset
#     :return: Keras model built, but not trained
#     """    

#     return 


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)
# Take one sequence (part of the training set)
# for trying out decoding.
for seq_index in range(3):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print(input_seq)
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]]
Input sentence: new jersey is sometimes quiet during autumn , and it is snowy in april .
Decoded sentence: la france est parfois froid en jais , mais il est parfois chaud en juile .t .a .in .ais .a paimon .ais . mais .e pai
-
[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]]]
Input sentence: the united states is usually chilly during july , and it is usually freezing in november .
Decoded sentence: la france est parfois froid en jais , mais il est parfois chaud en juile .t .a .in .ais .a paimon .ais . mais .e pai
-
[[[ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  [ 0.  0.  0. ...,  0.  0.  0.]
  ..., 
  [ 0

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [44]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    print(input_shape)

# -------------------------------------
    input_layer = Input(shape=input_shape[1:])
    rnn = Embedding(english_vocab_size, french_vocab_size, input_length=output_sequence_length)(input_layer)
    rnn = GRU(output_sequence_length, return_sequences=True)(rnn)
    rnn = Dense(french_vocab_size, activation='softmax')(rnn)


    model = Model(inputs=input_layer, outputs=rnn)
    model.compile(loss=sparse_categorical_crossentropy, optimizer=Adam(.001), metrics=['accuracy'])

# ------------------------------------
              
    return model
tests.test_model_final(model_final)


print('Final Model Loaded')
# # TODO: Train the final model
# # TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2],))

# TODO: Reshape the input
# tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2],))

print(tmp_x.shape)
# # # TODO: Train and Print prediction(s)
f_model = model_final( tmp_x.shape,max_french_sequence_length,english_vocab_size+1,french_vocab_size+1)
f_model.summary()
# # print(tmp_x.shape)
# # tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[-2],))
# # print(tmp_x.shape)

# # # TODO: Train the neural network
f_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=3, validation_split=0.2)

(137861, 15)
Final Model Loaded
(137861, 21)
(137861, 21)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_34 (InputLayer)        (None, 21)                0         
_________________________________________________________________
embedding_34 (Embedding)     (None, 21, 345)           69000     
_________________________________________________________________
gru_34 (GRU)                 (None, 21, 21)            23121     
_________________________________________________________________
dense_33 (Dense)             (None, 21, 345)           7590      
Total params: 99,711
Trainable params: 99,711
Non-trainable params: 0
_________________________________________________________________
Train on 110288 samples, validate on 27573 samples
Epoch 1/3
110288/110288 [==============================] - 11s 95us/step - loss: 4.3071 - acc: 0.4191 - val_loss: 3.2410 - val_acc: 0.4130
Epoch 2/3
110288/11028

In [13]:
index_to_words = {id: word for word, id in english_tokenizer.word_index.items()}
index_to_words[0] = ''

In [14]:
sentence = [index_to_words.get(int(Ids)) for Ids in tmp_x[:1][0] if index_to_words.get(int(Ids))  is not None]
print ("English Sentence: ",' '.join(sentence))

# Print prediction(s)
print("French Sentence:" , logits_to_text(f_model.predict(tmp_x[:1])[0], french_tokenizer))

English Sentence:  new jersey is sometimes quiet during autumn and it is snowy in april        
French Sentence: new jersey est parfois calme en l' et il est est en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Prediction (IMPLEMENTATION)

In [46]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    
    print("initial x:",x.shape)
    print("initial y:",y.shape)
    
    max_english_sequence_length = x.shape[1]
    max_french_sequence_length = y.shape[1]
    english_vocab_size = len(x_tk.word_index)
    french_vocab_size = len(y_tk.word_index)
    
    print('Data Preprocessed')
    print("Max English sentence length:", max_english_sequence_length)
    print("Max French sentence length:", max_french_sequence_length)
    print("English vocabulary size:", english_vocab_size)
    print("French vocabulary size:", french_vocab_size)


    tmp_x = pad(x, max_english_sequence_length)
    print("x pad with max_english_sequence_length:",tmp_x.shape)
    tmp_x = tmp_x.reshape((-1, x.shape[-1],))
    print("x reshape with x.shape[-2]:",tmp_x.shape,"x.shape[-1]=",x.shape[-1])

    model = model_final(tmp_x.shape, max_english_sequence_length , english_vocab_size + 1, french_vocab_size + 1)
    model.summary()
    model.fit(tmp_x, y, batch_size=1024, epochs=1, validation_split=0.2)
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences,len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))

final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

initial x: (137861, 15)
initial y: (137861, 21, 1)
Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344
x pad with max_english_sequence_length: (137861, 15)
x reshape with x.shape[-2]: (137861, 15) x.shape[-1]= 15
(137861, 15)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_36 (InputLayer)        (None, 15)                0         
_________________________________________________________________
embedding_36 (Embedding)     (None, 15, 345)           69000     
_________________________________________________________________
gru_36 (GRU)                 (None, 15, 15)            16245     
_________________________________________________________________
dense_35 (Dense)             (None, 15, 345)           5520      
Total params: 90,765
Trainable params: 90,765
Non-trainable params: 0
________________________

ValueError: Error when checking target: expected dense_35 to have shape (None, 15, 1) but got array with shape (137861, 21, 1)

## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

In [ ]:
!!python -m nbconvert *.ipynb

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?